In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv
from tqdm import tqdm
tqdm.pandas()
%load_ext autoreload
%autoreload 2

In [2]:
# change dir to ml repo at ml/ root (if you started in notebook/)
os.chdir('../')
print(Path.cwd())
load_dotenv()

/home/opadmin/llama2d_opentable/lm_act_eval


True

In [3]:
from datasets import Dataset
import pandas as pd
from lm_act_eval.evaluation_harness.helper_functions.multion import (
  action_prefix,
  clean_extracted_text,
  extract_thought,
  extract_action,
  extract_explanation,
  ParseChatCompletion,
  extract_first
)
from lm_act_eval.evaluation_harness.utils.url import is_screenshot_url_accessible

/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Google Cloud not set up, skipping import of providers.gemini_utils.generate_from_gemini_completion
Google Cloud not set up, skipping import of vertexai.preview.generative_models.Image and llms.generate_from_gemini_completion


/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/deepeval/__init__.py:41: UserWarning: You are using deepeval version 0.20.90, however version 0.21.23 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [4]:
from typing import Callable
from tqdm import tqdm

In [5]:
# subdependent 
from lm_act_eval.evaluation_harness.evaluators.sft.dataframe import DataFrameEvaluator
from lm_act_eval.evaluation_harness.helper_functions import multion

In [6]:
from lm_act_eval._init_conf import init_conf
hydra_cfg = init_conf('trajectory_eval')
cfg = hydra_cfg.eval

/home/opadmin/llama2d_opentable/lm_act_eval/lm_act_eval/_init_conf.py:9: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="../config")


Composed Configurations:
project: opentable
eval:
  sft:
    trajectory:
      data:
        path: lm_act_eval/.cache/five-star-trajectories/csv/data+gptv-eligible.csv
        columns:
          'y': ground_truth
          y_': GPTV_generations
        extract_fs:
          QUERY:
            QUERY: null
          screenshot:
            screenshot: null
          GOAL:
            chat_completion_messages: parse_completion.parse_content
        logging:
          wandb:
            project: opentable
            result: lm_act_eval-run
          braintrust:
            project: multion_opentable
      comparator:
        gpt-v:
          model: gpt-4-vision-preview
          max_token: 300
          img_fidelity: high
      metrics:
        gpt-v:
          inputs:
          - GOAL
          - QUERY
          - screenshot
          args: null
        llm_relevancy:
        - explanation



In [7]:
df = pd.read_csv(cfg.sft.trajectory.data.path)
entry = df.iloc[0]
print(
  df.chat_completion_messages.iloc[0][:100],
  df.columns
)

[{'role': 'system', 'content': '**Goal**: Let\'s play a game - You are an expert agent named MULTI·O Index(['Unnamed: 0', 'session_id', 'idx_in_session', '_id', 'event_id',
       'parent_id', 'event_type', 'event_name', 'config', 'children_ids',
       'inputs', 'outputs', 'error', 'start_time', 'end_time', 'metadata',
       'feedback', 'metrics', 'user_properties', 'source', 'duration',
       'project_id', 'tenant', 'QUERY', 'URL', 'DOM', 'RULES', 'USER_CONTEXT',
       'PREV_ACTIONS', 'chat_history', 'CURRENT_TIME', 'planPrompt',
       'previousActionsRepetitionPrompt', 'chat_completion_messages',
       'ground_truth', 'screenshot', 'prompt', 'internal_metadata',
       'chat_completion_messages_content', 'GPTV_generations'],
      dtype='object')


In [8]:
from lm_act_eval.evaluation_harness.evaluators.sft.trajectory import TableTrajectoryEvaluator

df_evaluator = TableTrajectoryEvaluator(cfg.sft.trajectory)

/home/opadmin/llama2d_opentable/lm_act_eval/lm_act_eval/evaluation_harness/helper_functions/multion.py:138: UserWarning: Failed to parse json, returning content field
  warnings.warn(f"Failed to parse json, returning content field")
/home/opadmin/llama2d_opentable/lm_act_eval/lm_act_eval/evaluation_harness/helper_functions/multion.py:156: UserWarning: Unexpected format, returning suspicious content of type <class 'str'>, using as is
  warnings.warn(f"Unexpected format, returning suspicious content of type {type(chat_completion_msgs)}, using as is")


In [9]:
df_evaluator.df.head(2)

,QUERY,screenshot,GOAL
169,What was the actual enrollment count of the cl...,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...
170,What was the actual enrollment count of the cl...,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...


In [10]:
hydra_cfg.eval.sft.trajectory.metrics

{'gpt-v': {'inputs': ['GOAL', 'QUERY', 'screenshot'], 'args': None}, 'llm_relevancy': ['explanation']}

In [33]:
from cmd import PROMPT
from lm_act_eval.evaluation_harness.evaluators import metric_registry
from deepeval.test_case import LLMTestCase

print(metric_registry.list_registered())

cp = metric_registry.get('contextual_precision')(threshold=0.7, model="gpt-4", include_reason=True)
cp

['edit_distance', 'contextual_precision', 'gpt-v']


In [35]:
result = cp(
  input=entry.chat_completion_messages_content,
  y_pred=entry.GPTV_generations,
  y=entry.ground_truth,
  query=[entry.QUERY]
)

/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Event loop is already running. Applying nest_asyncio patch to allow async execution...

/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [36]:
print(cp.score, cp.explanation)

1.0 The score is 1.00, indicating perfect precision. This is due to the fact that the node in the retrieval context, which is ranked first, is directly relevant to the user's objective. The node provides useful information to formulate the action plan, as clearly explained in its reason field. Therefore, the score is at its maximum as the relevant information is ranked at the top.


## Save/Load Data

In [ ]:
# traj_df_new = traj_df.merge(eligible_traj_df, how='left')
# traj_df_new.to_csv(dataset_path/'csv/data+gptv_eval.csv')

In [ ]:
table_name = 'data+gptv_eval-eligible'
artifact_name = 'gptv_opentable-trajectory_eval'
eligible_traj_df.to_csv(dataset_path/f'csv/{table_name}.csv')

In [ ]:
eligible_traj_df = pd.read_csv(
  dataset_path/f'csv/{table_name}.csv', index_col=0)
eligible_traj_df.index.name = None
eligible_traj_df.head(2)

,session_id,idx_in_session,_id,event_id,parent_id,event_type,event_name,config,children_ids,inputs,...,planPrompt,previousActionsRepetitionPrompt,chat_completion_messages,ground_truth,screenshot,prompt,internal_metadata,GPTV response,chat_completion_messages_content,GPTV_evaluation
169,36c2f0ba-e215-4081-96b4-ecb04c10a517,0,65ed4fc8166e07fe3c29c476,6c8932f3-c31d-4683-aeb0-52a5bbe5a978,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",PLAN:\n1. Use the Google search function to fi...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,It appears that you are looking for informatio...,**Goal**: Let's play a game - You are an exper...,SCORE: 60\nEXPLANATION:\nThe user appears to b...
170,36c2f0ba-e215-4081-96b4-ecb04c10a517,1,65ed4fd1b2b5b0c1673c47d7,172ff25e-efdb-4002-b43a-0180f3f0eb19,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",COMMANDS:\nCLICK 26\nEXPLANATION: I am clickin...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,The image shows a webpage from ClinicalTrials....,**Goal**: Let's play a game - You are an exper...,SCORE: 95\nEXPLANATION:\nThe user is very clos...


Logging

In [ ]:
# log data
import wandb
from dotenv import load_dotenv
load_dotenv()
wandb.login(relogin=True)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/opadmin/.netrc


True

In [ ]:
opentable_artifact = wandb.Artifact("opentable_trajectories-gptv_eval", type="dataset")
# opentable_table = wandb.Table(dataframe=traj_df_new)
opentable_table_eligible = wandb.Table(dataframe=eligible_traj_df)
#
# opentable_artifact.add(opentable_table, "opentable")
opentable_artifact.add(
  opentable_table_eligible, artifact_name)
# opentable_artifact.add_file(str(dataset_path/'csv/data+gptv.csv'))
opentable_artifact.add_file(str(dataset_path/f'csv/{table_name}.csv'))


In [ ]:
# Start a W&B run to log data
run = wandb.init(project="trajectory_eval", reinit=True)
# Log the table to visualize with a run...
run.log({"opentable_gptv": opentable_table_eligible})

# and Log as an Artifact to increase the available row limit!
run.log_artifact(opentable_artifact)

wandb: Currently logged in as: mmym (multion-agi). Use `wandb login --relogin` to force relogin


wandb: WARNING Serializing object of type str that is 531752 bytes
wandb: WARNING Serializing object of type str that is 517908 bytes
wandb: WARNING Serializing object of type str that is 541284 bytes
wandb: WARNING Serializing object of type str that is 525810 bytes
wandb: WARNING Serializing object of type str that is 116864 bytes
wandb: WARNING Serializing object of type str that is 121274 bytes
wandb: WARNING Serializing object of type str that is 120688 bytes
wandb: WARNING Serializing object of type str that is 124364 bytes
wandb: WARNING Serializing object of type str that is 127354 bytes
wandb: WARNING Serializing object of type str that is 129308 bytes
wandb: WARNING Serializing object of type str that is 129582 bytes
wandb: WARNING Serializing object of type str that is 135310 bytes
wandb: WARNING Serializing object of type str that is 137856 bytes
wandb: WARNING Serializing object of type str that is 137644 bytes
wandb: WARNING Serializing object of type str that is 136800 b

<Artifact opentable_trajectories-gptv_eval>